Get help from: https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/

https://towardsdatascience.com/simple-bert-using-tensorflow-2-0-132cb19e9b22



In [1]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [0]:
import pandas as pd
downloaded = drive.CreateFile({'id':'1UDiOho3fJT_VRssVjn9ZI9SrBI34Fkh9'}) ## Load Secondary_Source.csv in the edge_list_creation folder
downloaded.GetContentFile('secondary_source.csv')  
file_dictionary = pd.read_csv('secondary_source.csv')
del file_dictionary['Unnamed: 0']

In [3]:
!pip install pytorch-pretrained-bert
from pytorch_pretrained_bert import BertTokenizer, BertModel

     |████████████████████████████████| 133kB 3.4MB/s 


In [0]:
import torch

In [0]:
data = file_dictionary.values

In [0]:
data = data[:150]

In [0]:
content = [v for i, v in enumerate(data)]

In [0]:
def get_ready(sentence):
  tokens=tokenizer.tokenize(sentence)
  tokens = ['[CLS]'] + tokens + ['[SEP]']
  T=100
  padded_tokens=tokens +['[PAD]' for _ in range(T-len(tokens))]
  attn_mask=[ 1 if token != '[PAD]' else 0 for token in padded_tokens]
  seg_ids=[0 for _ in range(len(padded_tokens))]
  padded_tokens = padded_tokens[:150]
  attn_mask = attn_mask[:150]
  seg_ids = seg_ids[:150]
  sent_ids=tokenizer.convert_tokens_to_ids(padded_tokens)
  token_ids = torch.tensor(sent_ids).unsqueeze(0) 
  attn_mask = torch.tensor(attn_mask).unsqueeze(0) 
  seg_ids = torch.tensor(seg_ids).unsqueeze(0)
  return token_ids, attn_mask, seg_ids

In [13]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

100%|██████████| 407873900/407873900 [00:05<00:00, 79122598.64B/s]


In [14]:
dic = {}
for i in range(len(content)):
  name = content[i][0]
  print("working on doc number "+ str(i))
  token_ids, attn_mask, seg_ids= get_ready(content[i][1])
  hidden_reps, cls_head = bert_model(token_ids, attention_mask = attn_mask,token_type_ids = seg_ids)
  vec = cls_head
  dic[name] = vec

working on doc number 0
working on doc number 1
working on doc number 2
working on doc number 3
working on doc number 4
working on doc number 5
working on doc number 6
working on doc number 7
working on doc number 8
working on doc number 9
working on doc number 10
working on doc number 11
working on doc number 12
working on doc number 13
working on doc number 14
working on doc number 15
working on doc number 16
working on doc number 17
working on doc number 18
working on doc number 19
working on doc number 20
working on doc number 21
working on doc number 22
working on doc number 23
working on doc number 24
working on doc number 25
working on doc number 26
working on doc number 27
working on doc number 28
working on doc number 29
working on doc number 30
working on doc number 31
working on doc number 32
working on doc number 33
working on doc number 34
working on doc number 35
working on doc number 36
working on doc number 37
working on doc number 38
working on doc number 39
working on

In [0]:
from scipy import spatial
def similar(a, b):
  a = a.detach().numpy()
  b = b.detach().numpy()
  result = 1 - spatial.distance.cosine(a, b)
  return result

In [0]:
threshold = 0.9 
Source = []
Target = []
Weight = []

In [21]:
for i in range(150):
  for k in range(i+1, 150):
    print("working on " + str(i) + " comparing "+ str(k))
    name_one = list(dic)[i]
    name_two = list(dic)[k]
    score = similar(dic[name_one],dic[name_two])
    if score > threshold:
      Source.append(name_one)
      Target.append(name_two)
      Weight.append(score) 

working on 0 comparing 1
working on 0 comparing 2
working on 0 comparing 3
working on 0 comparing 4
working on 0 comparing 5
working on 0 comparing 6
working on 0 comparing 7
working on 0 comparing 8
working on 0 comparing 9
working on 0 comparing 10
working on 0 comparing 11
working on 0 comparing 12
working on 0 comparing 13
working on 0 comparing 14
working on 0 comparing 15
working on 0 comparing 16
working on 0 comparing 17
working on 0 comparing 18
working on 0 comparing 19
working on 0 comparing 20
working on 0 comparing 21
working on 0 comparing 22
working on 0 comparing 23
working on 0 comparing 24
working on 0 comparing 25
working on 0 comparing 26
working on 0 comparing 27
working on 0 comparing 28
working on 0 comparing 29
working on 0 comparing 30
working on 0 comparing 31
working on 0 comparing 32
working on 0 comparing 33
working on 0 comparing 34
working on 0 comparing 35
working on 0 comparing 36
working on 0 comparing 37
working on 0 comparing 38
working on 0 comparin

In [0]:
result = pd.DataFrame({'Source':Source, 'Target':Target, 'Weight':Weight})

In [23]:
result

,Source,Target,Weight
0,Llop_2010_AuOr_28_Rec._OBO_160_5.txt,Dercksen_1997_ArAn3_BilgicFS_ikribu.txt,0.949883
1,Llop_2010_AuOr_28_Rec._OBO_160_5.txt,Günbatti_2008_AOF 35-1_An Eponym List - KEL G...,0.932001
2,Llop_2010_AuOr_28_Rec._OBO_160_5.txt,Goetze_1937_JAOS57_Subartu.txt,0.962704
3,Llop_2010_AuOr_28_Rec._OBO_160_5.txt,p249_s_a.txt,0.920252
4,Llop_2010_AuOr_28_Rec._OBO_160_5.txt,OzgucN_1994_HroudaFS_seals1a-b.txt,0.929565
...,...,...,...
4555,rosen1998.txt,owen1990.txt,0.980389
4556,rosen1998.txt,Donbaz_1984_Kienast Kaufvertragsrecht_review.txt,0.909389
4557,nougayrol1960.txt,owen1990.txt,0.975669
4558,nougayrol1960.txt,Donbaz_1984_Kienast Kaufvertragsrecht_review.txt,0.931222


In [24]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


In [27]:
result.to_csv('BERT_edge_list2.csv', index=False)
!cp BERT_edge_list2.csv drive/MyDrive/BerkeleyDH/AWCA/Collab_notebooks/edge_list_creation/

cp: cannot create regular file 'drive/MyDrive/BerkeleyDH/AWCA/Collab_notebooks/edge_list_creation/': No such file or directory
